<a href="https://colab.research.google.com/github/d7lewandowski/pyspark-script/blob/main/pyspark_training_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=bfc5ceaeeb525ba9555d7fe3b66d51039a121002bc6a03ab2588f8ff81327ff1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# SQL-style and SQL function on a DataFrame
RDD and DataFrame

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [ ]:
# Create a SparkSession
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()

In [ ]:
lines = spark.sparkContext.textFile('./drive/MyDrive/Taming Big Data with Apache Spark and Python - Hands On!/Data/fakefriends.csv')

In [ ]:
lines.collect()[:3]

['0,Will,33,385', '1,Jean-Luc,26,2', '2,Hugh,55,221']

In [ ]:
def mapper(line):
  fields = line.split(',')
  return Row(ID = int(fields[0]), name = str(fields[1]), age = int(fields[2]), numFriends = int(fields[3]))

people = lines.map(mapper)

In [ ]:
people.collect()[:3]

[Row(ID=0, name='Will', age=33, numFriends=385),
 Row(ID=1, name='Jean-Luc', age=26, numFriends=2),
 Row(ID=2, name='Hugh', age=55, numFriends=221)]

In [ ]:
schemaPeople = spark.createDataFrame(people).cache()
schemaPeople.createOrReplaceTempView('people')

In [ ]:
teenagers = spark.sql('SELECT * FROM people WHERE age >= 13 AND age <= 19')

In [ ]:
for teen in teenagers.collect():
  print(teen)

Row(ID=21, name='Miles', age=19, numFriends=268)
Row(ID=52, name='Beverly', age=19, numFriends=269)
Row(ID=54, name='Brunt', age=19, numFriends=5)
Row(ID=106, name='Beverly', age=18, numFriends=499)
Row(ID=115, name='Dukat', age=18, numFriends=397)
Row(ID=133, name='Quark', age=19, numFriends=265)
Row(ID=136, name='Will', age=19, numFriends=335)
Row(ID=225, name='Elim', age=19, numFriends=106)
Row(ID=304, name='Will', age=19, numFriends=404)
Row(ID=341, name='Data', age=18, numFriends=326)
Row(ID=366, name='Keiko', age=19, numFriends=119)
Row(ID=373, name='Quark', age=19, numFriends=272)
Row(ID=377, name='Beverly', age=18, numFriends=418)
Row(ID=404, name='Kasidy', age=18, numFriends=24)
Row(ID=409, name='Nog', age=19, numFriends=267)
Row(ID=439, name='Data', age=18, numFriends=417)
Row(ID=444, name='Keiko', age=18, numFriends=472)
Row(ID=492, name='Dukat', age=19, numFriends=36)
Row(ID=494, name='Kasidy', age=18, numFriends=194)


In [ ]:
schemaPeople.groupBy('age').count().orderBy('age').show()

+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
| 23|   10|
| 24|    5|
| 25|   11|
| 26|   17|
| 27|    8|
| 28|   10|
| 29|   12|
| 30|   11|
| 31|    8|
| 32|   11|
| 33|   12|
| 34|    6|
| 35|    8|
| 36|   10|
| 37|    9|
+---+-----+
only showing top 20 rows



In [ ]:
spark.stop()

# Using DataFrames instead of RDD's

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()

In [ ]:
people = spark.read.option('header', 'true').option('inferSchema', 'true').csv(r'./drive/MyDrive/Taming Big Data with Apache Spark and Python - Hands On!/Data/fakefriends-header.csv')

In [ ]:
people.show()

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     0|    Will| 33|    385|
|     1|Jean-Luc| 26|      2|
|     2|    Hugh| 55|    221|
|     3|  Deanna| 40|    465|
|     4|   Quark| 68|     21|
|     5|  Weyoun| 59|    318|
|     6|  Gowron| 37|    220|
|     7|    Will| 54|    307|
|     8|  Jadzia| 38|    380|
|     9|    Hugh| 27|    181|
|    10|     Odo| 53|    191|
|    11|     Ben| 57|    372|
|    12|   Keiko| 54|    253|
|    13|Jean-Luc| 56|    444|
|    14|    Hugh| 43|     49|
|    15|     Rom| 36|     49|
|    16|  Weyoun| 22|    323|
|    17|     Odo| 35|     13|
|    18|Jean-Luc| 45|    455|
|    19|  Geordi| 60|    246|
+------+--------+---+-------+
only showing top 20 rows



In [ ]:
print(people.printSchema())

root
 |-- userID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)

None


In [ ]:
people.select('name').show()

+--------+
|    name|
+--------+
|    Will|
|Jean-Luc|
|    Hugh|
|  Deanna|
|   Quark|
|  Weyoun|
|  Gowron|
|    Will|
|  Jadzia|
|    Hugh|
|     Odo|
|     Ben|
|   Keiko|
|Jean-Luc|
|    Hugh|
|     Rom|
|  Weyoun|
|     Odo|
|Jean-Luc|
|  Geordi|
+--------+
only showing top 20 rows



In [ ]:
people.filter(people.age <= 18).show()

+------+-------+---+-------+
|userID|   name|age|friends|
+------+-------+---+-------+
|   106|Beverly| 18|    499|
|   115|  Dukat| 18|    397|
|   341|   Data| 18|    326|
|   377|Beverly| 18|    418|
|   404| Kasidy| 18|     24|
|   439|   Data| 18|    417|
|   444|  Keiko| 18|    472|
|   494| Kasidy| 18|    194|
+------+-------+---+-------+



In [ ]:
people.groupBy('age').count().show()

+---+-----+
|age|count|
+---+-----+
| 31|    8|
| 65|    5|
| 53|    7|
| 34|    6|
| 28|   10|
| 26|   17|
| 27|    8|
| 44|   12|
| 22|    7|
| 47|    9|
| 52|   11|
| 40|   17|
| 20|    5|
| 57|   12|
| 54|   13|
| 48|   10|
| 19|   11|
| 64|   12|
| 41|    9|
| 43|    7|
+---+-----+
only showing top 20 rows



In [ ]:
people.select(people.name, (people.age + 10).alias('age')).show()

+--------+---+
|    name|age|
+--------+---+
|    Will| 43|
|Jean-Luc| 36|
|    Hugh| 65|
|  Deanna| 50|
|   Quark| 78|
|  Weyoun| 69|
|  Gowron| 47|
|    Will| 64|
|  Jadzia| 48|
|    Hugh| 37|
|     Odo| 63|
|     Ben| 67|
|   Keiko| 64|
|Jean-Luc| 66|
|    Hugh| 53|
|     Rom| 46|
|  Weyoun| 32|
|     Odo| 45|
|Jean-Luc| 55|
|  Geordi| 70|
+--------+---+
only showing top 20 rows



In [ ]:
spark.stop()

# Task 1

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func

In [ ]:
spark = SparkSession.builder.appName('Sql-spark').getOrCreate()

In [ ]:
people = spark.read.option('header', 'true').option('inferSchema', 'true').csv(r'./drive/MyDrive/Taming Big Data with Apache Spark and Python - Hands On!/Data/fakefriends-header.csv')

In [ ]:
people.show()

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     0|    Will| 33|    385|
|     1|Jean-Luc| 26|      2|
|     2|    Hugh| 55|    221|
|     3|  Deanna| 40|    465|
|     4|   Quark| 68|     21|
|     5|  Weyoun| 59|    318|
|     6|  Gowron| 37|    220|
|     7|    Will| 54|    307|
|     8|  Jadzia| 38|    380|
|     9|    Hugh| 27|    181|
|    10|     Odo| 53|    191|
|    11|     Ben| 57|    372|
|    12|   Keiko| 54|    253|
|    13|Jean-Luc| 56|    444|
|    14|    Hugh| 43|     49|
|    15|     Rom| 36|     49|
|    16|  Weyoun| 22|    323|
|    17|     Odo| 35|     13|
|    18|Jean-Luc| 45|    455|
|    19|  Geordi| 60|    246|
+------+--------+---+-------+
only showing top 20 rows



In [ ]:
people.select('age', 'friends').groupBy('age').avg('friends').sort('age').show()

+---+------------------+
|age|      avg(friends)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
| 23|             246.3|
| 24|             233.8|
| 25|197.45454545454547|
| 26|242.05882352941177|
| 27|           228.125|
| 28|             209.1|
| 29|215.91666666666666|
| 30| 235.8181818181818|
| 31|            267.25|
| 32| 207.9090909090909|
| 33| 325.3333333333333|
| 34|             245.5|
| 35|           211.625|
| 36|             246.6|
| 37|249.33333333333334|
+---+------------------+
only showing top 20 rows



In [ ]:
people.select('age', 'friends').groupBy('age').agg(func.round(func.avg('friends'), 2).alias('avg_num_of_friends_by_age')).sort('age').show()

+---+-------------------------+
|age|avg_num_of_friends_by_age|
+---+-------------------------+
| 18|                   343.38|
| 19|                   213.27|
| 20|                    165.0|
| 21|                   350.88|
| 22|                   206.43|
| 23|                    246.3|
| 24|                    233.8|
| 25|                   197.45|
| 26|                   242.06|
| 27|                   228.13|
| 28|                    209.1|
| 29|                   215.92|
| 30|                   235.82|
| 31|                   267.25|
| 32|                   207.91|
| 33|                   325.33|
| 34|                    245.5|
| 35|                   211.63|
| 36|                    246.6|
| 37|                   249.33|
+---+-------------------------+
only showing top 20 rows



In [ ]:
spark.stop()